In [ ]:
import os
import sys
from pathlib import Path
import gc

import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

sys.path.append(str(Path.cwd().resolve().parent))

In [ ]:
def cleanup_model(model, tokenizer):
    # Delete model and tokenizer
    if model is not None:
        del model
    if tokenizer is not None:
        del tokenizer
    
    # Force garbage collection
    gc.collect()
    
    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
    print(f"GPU memory after cleanup: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

In [ ]:
from src.config import (
    MODELS_DIR,
    GSM8K_PATH,
    TEACHER_SYSTEM_PROMPT,
    TEACHER_USER_PROMPT,
)
from src.dataset_generator.helpers.answers import (
    ParsingError,
    parse_gold_answer_number,
    parse_teacher_final_answer,
)

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
def build_prompt_cot(question: str) -> str:
    sys_txt = TEACHER_SYSTEM_PROMPT.strip()
    usr_txt = TEACHER_USER_PROMPT.strip().format(question=question.strip())
    return f"{sys_txt}\n\n{usr_txt}\n"


def build_prompt_label_only(question: str) -> str:
    prompt = f"""You are a rigorous and precise math solver. Solve the question and output the result only as:
Final Answer: <number>

Example 1:
Question: A farm has 3 barns with 12 cows each. It sells 7 cows and buys 5 more. How many cows now?
Final Answer: 34

Example 2:
Question: Pens cost $2 and notebooks $5. Alex buys 3 pens and 2 notebooks and pays with $20. How much change?
Final Answer: 4

Example 3:
Question: A tank holds 250 liters. 35% is drained, then 40 liters are added. How many liters now?
Final Answer: 202.5

Question: {question.strip()}"""
        
    return prompt

In [ ]:
def create_quantization_config(bf16: bool) -> BitsAndBytesConfig:
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if bf16 else torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )


def load_model(model_id: str, load_path: str | None, quant_config: BitsAndBytesConfig):
    return AutoModelForCausalLM.from_pretrained(
        load_path if load_path else model_id,
        quantization_config=quant_config,
        device_map="auto",
    )


def setup_tokenizer(model_id: str, load_path: str | None, model):
    tokenizer = AutoTokenizer.from_pretrained(
        load_path if load_path else model_id, 
        use_fast=True, 
        padding_side="left", 
        truncation_size="left"
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.generation_config.pad_token_id = tokenizer.pad_token_id

    return tokenizer


def load_model_and_tokenizer(model_id: str, bf16: bool, load_path: str | None = None):
    quant_config = create_quantization_config(bf16)
    model = load_model(model_id, load_path, quant_config)
    tokenizer = setup_tokenizer(model_id, load_path, model)
    return model, tokenizer

In [ ]:
def prepare_batch_inputs(questions: list[str], mode: str, tokenizer):
    build_prompt = build_prompt_cot if mode == "cot" else build_prompt_label_only
    prompts = [build_prompt(q) for q in questions]
    
    return tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
    ), prompts

def extract_responses(generated_tokens, prompts, tokenizer):
    full_texts = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return [full[len(prompt):].strip() for prompt, full in zip(prompts, full_texts)]


def process_batch(model, tokenizer, questions: list[str], mode: str) -> list[str]:
    encoded_inputs, prompts = prepare_batch_inputs(questions, mode, tokenizer)
    encoded_inputs = encoded_inputs.to(model.device)
    
    generated_tokens = model.generate(
        **encoded_inputs,
        do_sample=False,
        use_cache=True,
    )
    return extract_responses(generated_tokens, prompts, tokenizer)


def extract_questions_and_answers(dataset) -> tuple[list[str], list[float | None]]:
    questions = dataset["question"]
    gold_nums = [parse_gold_answer_number(answer) for answer in dataset["answer"]]
    return questions, gold_nums


def load_benchmark_dataset(
    path: str, split: str, limit: int | None = None
) -> tuple[list[str], list[float | None]]:
    dataset = load_dataset(path, name="main", split=split)
    if limit is not None:
        dataset = dataset.select(range(min(limit, len(dataset))))
        
    return extract_questions_and_answers(dataset)


def count_correct_predictions(predictions: list[str], gold_answers: list[float | None]) -> int:
    correct_count = 0
    
    for pred_text, gold_num in zip(predictions, gold_answers):
        try:
            pred_num = parse_teacher_final_answer(pred_text)
        except ParsingError:
            pred_num = None
        
        if (pred_num is not None and 
            gold_num is not None and 
            pred_num == gold_num):
            correct_count += 1
            
    return correct_count

def evaluate_predictions(predictions: list[str], gold_answers: list[float | None]) -> dict:
    correct_count = count_correct_predictions(predictions, gold_answers)
    total_count = len(predictions)
    
    accuracy = correct_count / total_count if total_count > 0 else 0.0
    
    return {"accuracy": accuracy, "n": total_count}


def generate_predictions(model, tokenizer, questions: list[str], mode: str, batch_size: int) -> list[str]:
    outputs = []
    model.eval()
    
    with torch.inference_mode():
        for i in tqdm(range(0, len(questions), batch_size), desc=f"Evaluating ({mode}, greedy, bs={batch_size})"):
            batch_questions = questions[i:i + batch_size]
            batch_outputs = process_batch(model, tokenizer, batch_questions, mode)
            outputs.extend(batch_outputs)

    return outputs


def benchmark(
    model, tokenizer, mode: str, batch_size: int, limit: int | None = None
):
    questions, gold_answers = load_benchmark_dataset(GSM8K_PATH, "test", limit)
    predictions = generate_predictions(model, tokenizer, questions, mode, batch_size)
    return evaluate_predictions(predictions, gold_answers)

In [ ]:
MODEL_ID = "Qwen/Qwen2.5-3B"
SCTOD_PATH = MODELS_DIR / "qwen2.5_3b_sctod_lora"
LABELONLY_PATH = MODELS_DIR / "qwen2.5_3b_labelonly_lora"

# RUNS = [
#     {"name": "student_sctod", "mode": "cot", "path": SCTOD_PATH},
#     {"name": "student_label_only", "mode": "label-only", "path": LABELONLY_PATH},
#     {"name": "base_cot_prompting", "mode": "cot", "path": None},
#     {"name": "base_label_only", "mode": "label-only", "path": None},
# ]

RUNS = [
    {"name": f"student_label_only_checkpoint_{i}", "mode": "label-only", "path": str(path)}
    for i, path in enumerate(
        sorted(
            SCTOD_PATH.glob("checkpoint-*"),
            key=lambda path: int(str(path).split("checkpoint-")[-1]),
        ),
        start=1,
    )
]

In [ ]:
limit = 200
batch_size = 16

results = []
model, tokenizer = None, None

for run in RUNS:
    name = run["name"]
    mode = run["mode"]
    path = run["path"]
    
    if model is not None or tokenizer is not None:
        print("Cleaning up previous model...")
        cleanup_model(model, tokenizer)
    
    print(f"\n=== Loading {name} ({'qlora' if path else 'base'}) ===")
    
    model, tokenizer = load_model_and_tokenizer(
        model_id=MODEL_ID,
        bf16=True,
        load_path=path,
    )
    # Ensure eval-time cache is on (may have been disabled in training config)
    if hasattr(model, "config"):
        model.config.use_cache = True

    print(f"GPU memory after loading: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

    metrics = benchmark(
        model,
        tokenizer,
        mode=mode,
        limit=limit,
        batch_size=batch_size,
    )
    print(f"{name} -> accuracy: {metrics['accuracy']:.4f}")
    results.append((name, metrics))

# Final cleanup
cleanup_model(model, tokenizer)

print("\n------- Summary -------")
for name, m in results:
    print(f"{name:>24}: {m['accuracy']:.4f}")